### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
training_set = train_datagen.flow_from_directory('./drive/MyDrive/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 11808 images belonging to 27 classes.


In [8]:
test_set = test_datagen.flow_from_directory('./drive/MyDrive/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 3560 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [9]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [11]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [12]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [13]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [14]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [15]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [18]:
classifier.fit(training_set,
                  epochs = 10,
                  validation_data = test_set)

Epoch 1/10
1181/1181 [==============================] - 9948s 8s/step - loss: 2.6633 - accuracy: 0.1732 - val_loss: 1.6731 - val_accuracy: 0.4329
Epoch 2/10
1181/1181 [==============================] - 379s 321ms/step - loss: 1.8188 - accuracy: 0.3704 - val_loss: 1.3108 - val_accuracy: 0.5486
Epoch 3/10
1181/1181 [==============================] - 380s 321ms/step - loss: 1.5082 - accuracy: 0.4710 - val_loss: 1.1119 - val_accuracy: 0.6191
Epoch 4/10
1181/1181 [==============================] - 373s 316ms/step - loss: 1.3183 - accuracy: 0.5365 - val_loss: 1.0416 - val_accuracy: 0.6649
Epoch 5/10
1181/1181 [==============================] - 372s 315ms/step - loss: 1.1820 - accuracy: 0.5802 - val_loss: 0.9236 - val_accuracy: 0.7180
Epoch 6/10
1181/1181 [==============================] - 372s 315ms/step - loss: 1.0792 - accuracy: 0.6203 - val_loss: 0.9239 - val_accuracy: 0.7034
Epoch 7/10
1181/1181 [==============================] - 388s 329ms/step - loss: 1.0229 - accuracy: 0.6423 - val_lo

#### Saving the Model

In [19]:
model_json = classifier.to_json()
with open("model_new_4.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new_4.h5')
print('Weights saved')

Model Saved
Weights saved
